# Chroma

[![Index](https://img.shields.io/badge/Index-blue)](../index.ipynb)
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/digillia/Digillia-Colab/blob/main/tools/chroma.ipynb)

Chroma est une base de données vectorielle:

- https://www.trychroma.com/
- https://github.com/chroma-core/chroma

Ceci est une démonstration de l'usage de Chroma et de l'API Open AI pour poser des questions sur le Code Civil:

1. Le Code Civil est d'abord décomposé en documents correspondant aux articles de code.
2. Ces documents sont convertis en vecteurs selon le système de codage d'Open AI (embeddings).
3. Ces vecteurs sont stockés dans la base vectorielle Chroma pour permettre des recherche sémantiques.
4. La base est ensuite recherchée par similarité à une question pour retrouver les documens correspondant aux articles de code pertinents.
5. Ces documents sont finalement soumis à ChatGPT avec la question pour y répondre.

In [89]:
from IPython import get_ipython

# Supprimer les commentaires pour installer
# !pip3 install -q -U tqdm

# À installer dans tous les cas pour Google Colab
if 'google.colab' in str(get_ipython()):
  !pip3 install -q -U chromadb
  !pip3 install -q -U openai
  !pip3 install -q -U pypdf
  !pip3 install -q -U tiktoken

In [90]:
# Cette variable python est accessible depuis les commandes bash
work_directory = './chroma'

# Récupération des données pour Google Colab
if 'google.colab' in str(get_ipython()):
  !curl --create-dirs -O --output-dir $work_directory "https://raw.githubusercontent.com/digillia/Digillia-Colab/main/tools/chroma/code_civil.pdf"

In [91]:
# Fonction utilitaire pour afficher du texte sur plusieurs lignes
def word_wrap(string, n_chars=72):
    if len(string) < n_chars:
        return string
    else:
        return string[:n_chars].rsplit(' ', 1)[0] + '\n' + word_wrap(string[len(string[:n_chars].rsplit(' ', 1)[0])+1:], n_chars)

## Chargement de la Clé pour OpenAI

Il vous faut obtenir d'Open AI une clé pour exécuter ce notebook Jupyter. Vous pouvez consulter [Where do I find my API key?](https://help.openai.com/en/articles/4936850-where-do-i-find-my-api-key). Ensuite, le chargement se fait soit à partir de l'environnement (fichier `.env`), soit à partir des secrets de Google Colab.

In [92]:
import os

openai_api_key = None
if 'google.colab' in str(get_ipython()):
  from google.colab import userdata
  openai_api_key = userdata.get('OPENAI_API_KEY')
else:
  from dotenv import load_dotenv, find_dotenv
  _ = load_dotenv(find_dotenv()) # read local .env file
  openai_api_key  = os.environ['OPENAI_API_KEY']

## Création d'une Base Vectorielle Chroma

In [93]:
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction #, SentenceTransformerEmbeddingFunction

# Open AI
embedding_function = OpenAIEmbeddingFunction(
    api_key=openai_api_key,
    # model_name='text-embedding-3-small',
    model_name='text-embedding-ada-002',
    # model_name='text-embedding-3-large',
)

# Camembert ne semble pas fonctionner et
# all-MiniLM-L6-v2 n'est pas adapté au français
# embedding_function = SentenceTransformerEmbeddingFunction(
#   # model_name="sentence-transformers/all-MiniLM-L6-v2",
#   # model_name="dangvantuan/sentence-camembert-base",
#   model_name="dangvantuan/sentence-camembert-large",
# )

# Exécuter chroma en mémoire
client = chromadb.Client()
# client = chromadb.PersistentClient(path=work_directory)

# Création d'une collection
collection_name = "code_civil"
if collection_name in list(map(lambda col: col.name, client.list_collections())):
    client.delete_collection(collection_name)
    print(f"Collection supprimée: {collection_name}")
collection = client.create_collection(collection_name, embedding_function=embedding_function)

Collection supprimée: code_civil


## Ajout de Documents à la Base Chroma

Le fichier pdf est converti en chaînes de caractères.

In [94]:
import re
from pypdf import PdfReader

reader = PdfReader(f'{work_directory}/code_civil.pdf')
pdf_texts = [p.extract_text().strip() for p in reader.pages]
# Suppression des pages vides
pdf_texts = [text for text in pdf_texts if text]
# Suppression des pieds de pages 'Code civil - Dernière modification le 21 mai 2023 - Document généré le 08 janvier 2024'
pdf_texts = [re.sub(r'Code\scivil\s-\sDernière\smodification\sle\s21\smai\s2023\s-\sDocument\sgénéré\sle\s08\sjanvier\s2024', '', text) for text in pdf_texts]
# Affichage de la page 6
print(word_wrap(pdf_texts[6]))

Chapitre III : De l'examen des caractéristiques génétiques
d'une
personne et de l'identification d'une personne par ses
empreintes
génétiques
Article 16-10
 
I.-L'examen des caractéristiques
génétiques constitutionnelles d'une personne ne peut être entrepris
qu'à
des fins médicales ou de recherche scientifique. Il est subordonné
au consentement exprès de la personne,
recueilli par écrit
préalablement à la réalisation de l'examen.  
II.-Le consentement prévu
au I est recueilli après que la personne a été dûment informée :  
1°
De la nature de l'examen ;  
2° De l'indication de l'examen, s'il
s'agit de finalités médicales, ou de son objectif, s'il s'agit de
recherches
scientifiques ;  
3° Le cas échéant, de la possibilité que
l'examen révèle incidemment des caractéristiques génétiques
sans
relation avec son indication initiale ou avec son objectif initial
mais dont la connaissance permettrait à la
personne ou aux membres de
sa famille de bénéficier de mesures de prévention, y compris de 

Le texte est ensuite divisé en articles de code.

In [95]:
import re

single_text = '\n'.join(pdf_texts)
regex_patterns = [
  re.compile(r'^Livre\s'),
  re.compile(r'^Section\s'), 
  re.compile(r'^Titre\s'), # Attention au Titre préliminaire
  re.compile(r'^Chapitre\s'),
  re.compile(r'^Article\s+\d')
]

chunks = []
current_chunk = ''
max_length = 0

for line in single_text.split('\n'):
  for pattern in regex_patterns:
    if re.search(pattern, line):
      current_chunk = current_chunk.strip()
      current_length = len(current_chunk)
      if current_length > max_length:
        max_length = current_length
      chunks.append(current_chunk)
      current_chunk = ''
      break
  current_chunk += line + '\n'
chunks.append(current_chunk.strip())
print(f"Total chunks: {len(chunks)}")
print(f"Chunk max length: {max_length}")
print('\n\n' + word_wrap(chunks[6]))

Total chunks: 3384
Chunk max length: 5287


Article 5

Il est défendu aux juges de prononcer par voie de
disposition générale et réglementaire sur les causes qui leur
sont
soumises.


Les articles de code sont convertis en embeddings.

In [96]:
import tiktoken
from tqdm import tqdm

# Ces éléments sont adaptés à OpenAIEmbeddingFunction
encoder = tiktoken.get_encoding("cl100k_base")
token_count_limit = 512
char_count = 1000
k=1.75

# Commenter cette ligne pour intégrer tous les documents (compter une bonne heure)
# documents = chunks
documents = chunks[:100]

# Réduction des documents trop longs (choix simplificateur pour un démo)
def reduce_document(document):
    # Rechercher le prochain . à partir de char_count
    pos = document[char_count:].index('.') + char_count
    return document[:pos]

embeddings = []
i = 0
for document in tqdm(documents):
    try:
        token_count = len(encoder.encode(document))
        if token_count > token_count_limit:
            # Première tentative de réduction : on recherche le prochain . à partir de char_count
            document = reduce_document(document)
            token_count = len(encoder.encode(document))
            if token_count > token_count_limit:
                # Deuxième tentative de réduction: on coupe le document à k*char_count caractères
                document = document[:round(k*char_count)]
                token_count = len(encoder.encode(document))
                if token_count > token_count_limit:
                    raise Exception(f"Après tentative de réduction, le document {i} est encore trop long.")
        # Commenter les deux lignes suivantes pour tester rapidement la réduction sur tous les documents
        embedding = embedding_function([document])[0]
        embeddings.append(embedding)
    except Exception as e:
        print(i, e)
        continue
    finally:
        i+=1


100%|██████████| 100/100 [00:25<00:00,  3.99it/s]


In [97]:
ids = [str(i) for i in range(len(documents))]
collection.add(
    ids=ids,
    documents=documents,
    embeddings=embeddings,
    # metadatas=metadatas, # Considérer les livres, sections, titres, chapitres et articles
)
collection.count()

100

## Recherche Sémantique dans la Base Chroma

In [98]:
query = "Que dit le code sur la génétique?"

results = collection.query(query_texts=[query], n_results=5)
retrieved_documents = results['documents'][0]

for document in retrieved_documents:
    print(word_wrap(document))
    print(len(document))
    print('\n')

Article 16-13
 
Nul ne peut faire l'objet de discriminations en raison
de ses caractéristiques génétiques.
106


Article 16-4
 
Nul ne peut porter atteinte à l'intégrité de l'espèce
humaine.
 
Toute pratique eugénique tendant à l'organisation de la
sélection des personnes est interdite.
 
Est interdite toute
intervention ayant pour but de faire naître un enfant génétiquement
identique à une autre
personne vivante ou décédée.
 
Sans préjudice des
recherches tendant à la prévention, au diagnostic et au traitement des
maladies, aucune
transformation ne peut être apportée aux caractères
génétiques dans le but de modifier la descendance de la
personne.
542


Code civil
10


Chapitre III : De l'examen des caractéristiques génétiques
d'une
personne et de l'identification d'une personne par ses
empreintes
génétiques
141


Article 16-11
 
L'identification d'une personne par ses empreintes
génétiques ne peut être recherchée que :
 
1° Dans le cadre de mesures
d'enquête ou d'instruction diligenté

# Extension RAG

In [99]:
from openai import OpenAI

def rag(query, retrieved_documents, model="gpt-3.5-turbo"):
    information = "\n\n".join(retrieved_documents)
    messages = [
        {
            "role": "system",
            "content": "Vous êtes juriste. Vos utilisateurs vous posent des questions sur le droit français."
            "On vous montre la question et les informations pertinentes du code civil."
            "Répondez succintement à votre utilisateur en n'utilisant que ces informations."
        },
        {"role": "user", "content": f"Question: {query}. \n Information: {information}"}
    ]
    api = OpenAI(api_key=openai_api_key)    
    response = api.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content

output = rag(query=query, retrieved_documents=retrieved_documents)
print(word_wrap(output))

Le Code civil interdit toute discrimination basée sur les
caractéristiques génétiques et interdit toute pratique eugénique visant
à sélectionner des personnes. Il est également interdit de créer un
enfant génétiquement identique à une autre personne vivante ou décédée.
En ce qui concerne l'identification par empreintes génétiques, celle-ci
est autorisée dans le cadre d'enquêtes ou d'instructions judiciaires, à
des fins médicales, de recherche scientifique, pour l'identification de
personnes décédées inconnues et dans certains cas spécifiques prévus
par la loi. Tout prélèvement ou identification génétique nécessite le
consentement préalable et exprès de la personne concernée.
